In [39]:
import pandas as pd
import importlib

In [43]:
import basic_example2 as be
importlib.reload(be)

<module 'basic_example2' from '/home/julia/school/bakalarka/hospital_sim/basic_example2.py'>

In [53]:
nurses = []
for i in range(2):
    nurses.append(be.Nurse(i))
patients = []
for i in range(3):
    patients.append(be.Patient(i, i % len(nurses)))


In [54]:
requests = be.generate_requests(len(patients))
sim = be.Simulator(requests, nurses, patients)
df = pd.DataFrame(sim.simulate())

[(1.797046784500056, 0), (133.12442583418752, 0), (139.57329573345882, 2), (173.19363523573995, 1), (214.6643211671755, 1), (252.23493728067842, 1), (274.3001718614435, 0), (287.0983503377955, 2), (290.2817206742447, 1), (315.0240805625233, 0), (332.11994180269096, 0), (353.60210512823727, 2), (377.43625940442496, 1), (433.93980585349664, 2), (491.8866210289797, 2), (580.552541528729, 1), (714.8842026311354, 2), (732.6917969424778, 1), (762.0684156073638, 0), (773.9018184475946, 1), (807.9096022084833, 0), (885.3187114350646, 0), (900.0160089269581, 0), (908.5071184017594, 2), (927.5248680115739, 0), (931.3458642389759, 1), (943.2943838605288, 1), (996.0804742263338, 1), (1113.701899483324, 2), (1155.1208961334216, 0), (1227.1923532526828, 1), (1243.6917972377807, 2), (1269.9671723663432, 1), (1292.553887303436, 0), (1347.2939857081785, 2), (1383.412347079752, 1), (1481.0511184144511, 0), (1491.9845447360688, 2), (1509.566626027194, 0), (1547.99713929714, 2), (1566.1380379850598, 1), (

In [59]:
requests = [(1, 0), (25, 2), (26, 1)] #time, patient id
sim = be.Simulator(requests, nurses, patients)
df = pd.DataFrame(sim.simulate())

1 request from patient 0
1 nurse 0 chosen for patient 0
2 nurse 0 departing to patient 0
12 nurse 0 arrived at patient 0
25 request from patient 2
25 nurse 1 chosen for patient 2 because nurse 0 is busy
26 request from patient 1
26 all nurses busy, request pushed back to 42
26 nurse 1 departing to patient 2
32 nurse 0 dealt with request from patient 0
36 nurse 1 arrived at patient 2
42 request from patient 1
42 all nurses busy, request pushed back to 42
42 nurse 0 returned from patient 0
42 request from patient 1
42 nurse 0 chosen for patient 1 because nurse 1 is busy
43 nurse 0 departing to patient 1
53 nurse 0 arrived at patient 1
56 nurse 1 dealt with request from patient 2
66 nurse 1 returned from patient 2
73 nurse 0 dealt with request from patient 1
83 nurse 0 returned from patient 1


In [64]:
df

,time,type,patient_id,requested_nurse_id,chosen_nurse_id,pushed_back,nurse_id,nurse_phase,pushed_back_time
0,1,request,0,0.0,0.0,False,NaN,NaN,NaN
1,2,nurse_phase,0,NaN,NaN,NaN,0.0,departing,NaN
2,12,nurse_phase,0,NaN,NaN,NaN,0.0,arrived,NaN
3,25,request,2,0.0,1.0,False,NaN,NaN,NaN
4,26,request,1,1.0,NaN,True,NaN,NaN,42.0
5,26,nurse_phase,2,NaN,NaN,NaN,1.0,departing,NaN
6,32,nurse_phase,0,NaN,NaN,NaN,0.0,dealt with request,NaN
7,36,nurse_phase,2,NaN,NaN,NaN,1.0,arrived,NaN
8,42,request,1,1.0,NaN,True,NaN,NaN,42.0
9,42,nurse_phase,0,NaN,NaN,NaN,0.0,returned,NaN


In [50]:
#kolikrát sestra vyřizovala pacienta
def patients_per_nurse(nurse_id):
    nurse_df =  df[df["chosen_nurse_id"] == nurse_id]
    display(nurse_df["patient_id"].value_counts())

In [78]:
patients_per_nurse(0)

patient_id
0    1
1    1
Name: count, dtype: int64

In [77]:
#kolik toho sestra nachodila
def nurse_distance_walked(nurse_id):
    nurse_df = df[df["nurse_id"] == nurse_id]
    times = nurse_df["time"].to_list()
    assert(len(times) % 4 == 0)
    request_num = (len(times) / 4)
    walk_time_there = times[1] - times[0]
    walk_time_back = times[3] - times[2]
    walk_time = request_num * (walk_time_there + walk_time_back)
    # display(nurse_df)
    return walk_time

print(nurse_distance_walked(1))

20.0


In [ ]:
#jak dlouho pacient čekal